# DTSA 5510 Unsupervised Algorithms in Machine Learning Final Project
- Github repository: https://github.com/luisdi28ms/unsupervised-learning

In [3]:
import pandas as pd

# References
- Dataset
    - https://www.kaggle.com/datasets/paramvir705/airbnb-data/data